# Detection of Toxic Comments

Kaggle competition: [Toxic Comment Classification Challenge](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge)

In [1]:
seed = 333

## Import

In [2]:
import csv
from datetime import datetime
import string
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation,GlobalMaxPool1D,Bidirectional
from keras.layers.embeddings import Embedding
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score

/Users/mazhurin/keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load Data

In [3]:
def read_comments(filename = ''):
    comments = []
    labels = []
    ids = []

    with open (filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)

        header = True
        for row in csvReader:
            if header:
                header = False
                continue
            ids.append(row[0])
            s = row[1].translate(string.punctuation)
            comments.append(s)          
            if len(row) > 2:
                classes = []
                for i in range(2,8):
                    classes.append(row[i])
                labels.append(classes)

    ids = np.asarray(ids)
    X = np.asarray(comments)
    Y = np.asarray(labels, dtype=int)

    return ids, X, Y

In [4]:
id_train, X_train, Y_train = read_comments('./data/train.csv')
id_test, X_test, _ = read_comments('./data/test.csv')
labels = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

max_comment = 0
for i in range(X_train.shape[0]):                               
    l = len(X_train[i].lower().split())
    if max_comment < l:
        max_comment = l

num_classes = len(labels)

## Split Data

In [5]:
kfold = KFold(n_splits=5,random_state=seed)
# folds = list()
# for train_index, dev_index in kfold.split(X_train):
#     folds.append({
#         'train_index' : train_index,
#         'dev_index' : dev_index,     
#     })

## Load Embeddings

In [6]:
def read_embeddings(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [7]:
word_to_index, index_to_word, word_to_vec = read_embeddings('./glove/glove.twitter.27B.25d.txt')
word_to_vec["0.065581"] = [ 0.39605,  -0.96669,   0.23706,  -0.41379,  -0.97006,   0.16601,  -1.292,
 -0.58989,   0.11632,  -1.365,    -0.27939,  -0.57222,  -0.97108,  -0.56319,
 -0.015263, -0.70465,  -0.13867 ,  1.0702 ,  -0.25557  , 0.25122,  -0.87755,
  0.70999 ,  0.9118 ,  -0.30077, 0 ]


In [8]:
for r in X_train:
    words = r.split()
    for w in words:
        if w not in word_to_index:
            if 'fuck' in w:
                print(w)



fistfuckee.
fuck.
fuck.++You
motherfucker!!!
++fuck
fuck.
Assfuckers
muthafucker.
""fucking
++Muthafucka
fuck.
dumbfucks.
muthafucka!
mothafucka!
clusterfucked
""clusterfucked
motherfucker.
""fucking
fuck...Before
http://gawker.com/5636765/facebook-ceo-admits-to-calling-users-dumb-fucks+
http://www.theweek.co.uk/technology/14625/are-users-%E2%80%98dumb-fucks%E2%80%99-trusting-data-facebook+
http://tdh.me/zuckerberg-called-early-facebook-users-dumb-fucks-so-what/+
http://anphicle.com/en/they-trust-me-dumb-fucks-facebook-ceo-mark-zuckerberg/+
fucks"".
fucks.++In
Corpsefucking
Corpsefucking
Corpsefucking
Corpsefucking
changes++fuck
motherfucker""
dumass.++fuck
""fucking
fuckkk
""mindfuck""
fuck's
Mindfuck
fuck?
fuckface.
fuck.
fuckin'
motherfucker,
bitch.++fuck
lord/fuck
fuckersItalic
dumbfuck,
fuck,
fuck????
fucking-ass
++fuck
Merchan-fucking-dise
T-fucking-800.
fuck...damn
""fuck""
""fuck,""
""fuck""
""fuck""
""fucking"")
""fuck""
""fuck""
""fuck""
""fuck""
""fuck""
fuck.
'fuck'
""fuck.

fucker.
dumbfuck?
""fuck
""fuck
""fuck
""fuck
fucki9ng
++fuck
fucker!126.12.157.168
fuckstain.
fuckhead!
fuckturd!
fucks,
Motherfucking
sick....fuckkkkk
""fuck
fuckwit;++http://www.telegraph.co.uk/sport/othersports/boxing/11626770/James-DeGale-claims-super-middleweight-title-with-points-win-over-Andre-Dirrell-in-Boston.html++http://www.independent.co.uk/sport/general/boxing/andre-dirrell-vs-james-degale-on-saturday-degale-can-join-britains-great-list-of-supermiddleweight-champions-says-steve-bunce-10259295.html++FYI....boxing
fuck!!
'fuck'
fuck,
fuckyourself!!!
motherfucker!!!!
motherfucker...only
fuck!ng
fuck!ng
fuck!ng
fuck!ng
'fuck'.
'fuck'
'fuck'
fuckwad""
fuck.
fuckknuckle?
fuckknuckle?
motherfucker!
motherfuck,
""fucking""
""fucking""
dumbfuck.
fuck,
motherfucker!!!
motherfuckers?
""fuck
fuckass.
fuck,
fuck.
fuck.
fuckin,ill
""fuck
fuckin'
fuck.
fucked?
Motherfucker
fuck.
bullshit+fuck
fuckin'
fuck's
'fuck
fuckwad.
motherfucker""
fuckin'
fuckin'
fuckin'
fuckin't
""fuck
motherfuck

mum!++fuck
too++fuck
++fuck
++fuck
don't-give-a-fuck
(fucking
motherfucker?
motherfucker!!!!!!!!!!!!!
motherfucker.
fuck.
motherfucker.
fuck+Got
fuckin'
fuckin'
fuckin'
fuckin'
fuckin'
fuckin'
fuckin'
fuckin'
fuck+Got
fuckin'
fuckin'
fuckin'
fuckin'
fuckin'
fuckin'
fuckin'
fuckin'
fuck+Got
fuckin'
fuckin'
fuckin'
fuckin'
fuckin'
fuckin'
fuckin'
fuckin'
fuck+Got
fuckin'
fuckin'
fuckin'
fuckin'
fuckin'
fucked.
""fucking
fuckwit.
Motherfucking
Motherfucker.+Example:
Motherfucking
fuckstick.
fuck.
motherfucker.
fuck,
""fuck
""fucking""
""fucking""
""fuck
""fuck
fuck.
lord/fuck
""fuck
""fuck
""fuck
fucker!!!
what-so-fucking-ever!!!
dickhead....fuck
dickhead....fuck
dickhead....fuck
dickhead....fuck
dickhead....fuck
dickhead....fuck
fuckish
fucker!!
fuck?
(fuck
Motherfucking
fucked.
fuckin'
dumbfuck+The
fuck's
fuck!!!
{{unblock-auto|...}fuck
""fuck
fuck.
website.++fuck
""clusterfucked
motherfucker.
fuck-sucking
julianfuckton?
dog-fucking?
examples!+http://propagandapress.org/2007/02/21/fuck-

## Baseline model

This is naive model based on averaging all word embeddings in a comment.

In [39]:
def average_comment(comment, word_to_vec):
    """
    Converts a comment into a list of words. 
    Extracts the GloVe representation of each word
    and averages its value into a single vector encoding the meaning of the sentence.
    
    Arguments:
    comment -- string
    word_to_vec_map -- dictionary mapping every word in a vocabulary 
    into its n-dimensional vector representation
    
    Returns:
    avg -- average vector encoding information about the sentence, numpy-array of shape (n,)
    """
    words = comment[0].lower().split()
    avg = np.zeros(shape=(len(word_to_vec['apple']), ))
    for w in words:
        if w in word_to_vec:
            avg += word_to_vec[w]
    avg = avg / len(words)
    return avg

In [40]:
averages = list()
for c in X_train:
    averages.append(average_comment(c, word_to_vec))
averages = np.asarray(averages)

In [78]:
averages_test = list()
for c in X_test:
    averages_test.append(average_comment(c, word_to_vec))
averages_test = np.asarray(averages_test)

/Users/mazhurin/keras/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide


In [68]:
from sklearn.metrics import log_loss
from keras.backend import int_shape
import keras.backend as K
import tensorflow as tf
_EPSILON = K.epsilon()
    
def custom_loss(y_true, y_pred):
    losses = []
    y_pred = K.clip(y_pred, _EPSILON, 1.0-_EPSILON)
    num_classes = int_shape(y_pred)[1]
    for i in range(num_classes):
        losses.append(K.mean(-(
            y_true[:, i]*K.log(y_pred[:, i]) - (1-y_true[:, i])*K.log(1-y_pred[:, i])
        ), axis = -1))
    loss = tf.stack(losses)
    loss = K.mean(loss, axis=-1)
        
    return loss
    #return tf.convert_to_tensor(losses, dtype='float32')

def baseline_model(num_dim=25, num_labels=6):
    # create model
    model = Sequential()
    model.add(Dense(num_labels, input_shape=(num_dim,), kernel_initializer='normal', 
                    activation='sigmoid'))
    #model.add(Activation('sigmoid'))
    model.compile(
        #loss='binary_crossentropy', 
        loss=custom_loss,
        optimizer='adam', metrics=['accuracy'])
    return model

In [69]:
model = baseline_model()

In [70]:
model.fit(averages, Y_train, epochs=3, verbose=1)

Epoch 1/3
 35008/159571 [=====>........................] - ETA: 12s - loss: -7.2864 - acc: 0.0599

KeyboardInterrupt: 

In [64]:
model.predict(averages[3003:3005])

array([[1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.]], dtype=float32)

In [57]:
Y_train[3003:3005]

array([[0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0]])

In [71]:
np.random.seed(seed)
estimator_baseline = KerasRegressor(build_fn=baseline_model, 
                                    nb_epoch=1, batch_size=5, verbose=1)

In [72]:
results = cross_val_score(estimator_baseline, averages, Y_train, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/1
 42035/127656 [========>.....................] - ETA: 36s - loss: -14.1116 - acc: 0.8558

KeyboardInterrupt: 

In [73]:
model.predict(np.asarray([average_comment("Love You", word_to_vec)]))

array([[0.9999994 , 0.99999964, 0.9999995 , 0.99999964, 0.9999995 ,
        0.99999964]], dtype=float32)

In [59]:
labels

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [311]:
Y_train[0]

array([0, 0, 0, 0, 0, 0])

## RNN

In [8]:
def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    m = X.shape[0]                                   # number of training examples
    X_indices = np.zeros(shape=(m, max_len))
    for i in range(m):                               
        sentence_words = X[i].lower().split()
        if len(sentence_words) > max_len:
            print ("Too many words:", len(sentence_words))
            continue
        j = 0
        for w in sentence_words:
            if w in word_to_index:
                X_indices[i, j] = word_to_index[w]
            else :
                X_indices[i, j] = word_to_index['word']
            j = j + 1
    return X_indices

In [9]:
X_train_indices = sentences_to_indices(X_train, word_to_index, max_comment)
X_test_indices = sentences_to_indices(X_test, word_to_index, max_comment)

2015
2321
1667
1667


In [10]:
def pretrained_embedding_layer(word_to_vec, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 
    
    Arguments:
    word_to_vec -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_len = len(word_to_index) + 1                  
    emb_dim = word_to_vec["cucumber"].shape[0]  
    emb_matrix = np.zeros(shape=(vocab_len, emb_dim))
    
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec[word]

    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    return embedding_layer

In [24]:
def rnn_model(input_shape, num_classes, word_to_vec, word_to_index):
    """
    Create the model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec -- dictionary mapping every word in a vocabulary into its n-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary 

    Returns:
    model -- a model instance in Keras
    """
    sentence_indices = Input(shape=input_shape, dtype='float32')
    embedding_layer = pretrained_embedding_layer(word_to_vec, word_to_index)
    embeddings = embedding_layer(sentence_indices)   
    
    X = Bidirectional(LSTM(18, return_sequences = False))(embeddings)
    #X = GlobalMaxPool1D()(X)
    X = Dropout(rate=0.1)(X)
#     X = LSTM(8, return_sequences = False)(X)
#     X = Dropout(rate=0.2)(X)
    X = Dense(units=50, activation='relu')(X)
    X = Dropout(0.1)(X)
    X = Dense(units=num_classes, activation='sigmoid')(X)
    model = Model(inputs = [sentence_indices], outputs = X)
    
    return model

In [25]:
model_RNN = rnn_model((max_comment,), num_classes, word_to_vec, word_to_index)
model_RNN.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1411)              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 1411, 25)          29837875  
_________________________________________________________________
bidirectional_3 (Bidirection (None, 36)                6336      
_________________________________________________________________
dropout_7 (Dropout)          (None, 36)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 50)                1850      
_________________________________________________________________
dropout_8 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 306       
Total para

In [26]:
model_RNN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
model_RNN.fit(X_train_indices, Y_train, epochs = 3, batch_size = 128, shuffle=True, verbose=True)

Epoch 1/3
  3456/159571 [..............................] - ETA: 32:08 - loss: 0.1069 - acc: 0.9650

KeyboardInterrupt: 

## Testing

In [15]:
Y_train[3003:3005]

array([[0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0]])

In [28]:
model_RNN.predict(X_train_indices[3003:3005])

array([[0.2932739 , 0.03831418, 0.16043979, 0.01002212, 0.13886277,
        0.02675362],
       [0.5962145 , 0.09602976, 0.5305401 , 0.0326421 , 0.47383296,
        0.10741065]], dtype=float32)

In [31]:
model_RNN.predict(
    sentences_to_indices(
        np.asarray([
            "I will kill you, motherfucker"]), 
        word_to_index, max_comment
    )
)

array([[0.53518003, 0.0736991 , 0.40560737, 0.02496896, 0.4049595 ,
        0.073121  ]], dtype=float32)

## Submission

In [33]:
y = model_RNN.predict(X_test_indices)

In [107]:
result = pd.concat([
    pd.DataFrame(id_test, columns=['id']), 
    pd.DataFrame(y, columns=labels)], axis=1)
result = result.set_index('id')

In [110]:
result = result.fillna(0.0)

,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,
00001cee341fdb12,0.999972,0.999977,0.999979,0.999974,0.999985,0.999987
0000247867823ef7,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
00013b17ad220c46,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
00017563c3f7919a,0.999999,0.999999,0.999999,0.999999,0.999999,0.999999
00017695ad8997eb,0.999990,0.999990,0.999988,0.999991,0.999987,0.999990
0001ea8717f6de06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
00024115d4cbde0f,0.999998,0.999999,0.999999,0.999999,0.999999,0.999999
000247e83dcc1211,0.999999,0.999999,0.999999,0.999999,0.999999,0.999999
00025358d4737918,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [111]:
result.to_csv('./result/anton_base_{}.csv'.format(datetime.now()), header=True)

,index,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,00001cee341fdb12,0.999972,0.999977,0.999979,0.999974,0.999985,0.999987
1,1,0000247867823ef7,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,2,00013b17ad220c46,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,3,00017563c3f7919a,0.999999,0.999999,0.999999,0.999999,0.999999,0.999999
4,4,00017695ad8997eb,0.999990,0.999990,0.999988,0.999991,0.999987,0.999990
5,5,0001ea8717f6de06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
6,6,00024115d4cbde0f,0.999998,0.999999,0.999999,0.999999,0.999999,0.999999
7,7,000247e83dcc1211,0.999999,0.999999,0.999999,0.999999,0.999999,0.999999
8,8,00025358d4737918,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
9,9,00026d1092fe71cc,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [79]:
y = model.predict(averages_test)